1\. **Pandas DataFrame**

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every `x` BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [1]:
# If haven't downloaded it yet, please get the data file with wget
!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

--2023-12-08 23:41:56--  https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.69.18, 2620:100:6025:18::a27d:4512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.69.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/xvjzaxzz3ysphme/data_000637.txt [following]
--2023-12-08 23:41:56--  https://www.dropbox.com/s/raw/xvjzaxzz3ysphme/data_000637.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc4139b32bb3073bd68249653601.dl.dropboxusercontent.com/cd/0/inline/CJDCuBCyHg0e-qTCNn7eRsyevrZ87wm_6Bu4Um-lDVPxAXgimOOFW14aJ2oKAqSfJ8Tz_1Vl6B6kHgHnG1VQmMn_2eMMIXmei1Zs8Hvb5ZStMbU0gEtPyAYzUWz__X-tTQ8/file# [following]
--2023-12-08 23:41:57--  https://uc4139b32bb3073bd68249653601.dl.dropboxusercontent.com/cd/0/inline/CJDCuBCyHg0e-qTCNn7eRsyevrZ87wm_6Bu4Um-lDVPxAXgimOOFW14aJ2oKAqSfJ8Tz_1Vl6B6kHgHnG1VQmMn_2eMMIXmei1Zs8Hvb5ZStMbU

1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [2]:
import pandas as pd
N = 20000
df = pd.read_csv("./data/data_000637.txt",nrows=N) 

2\. Estimate the number of BX in a ORBIT (the value `x`).

*Hint*: check when the BX counter reaches the maximum value before being reset to 0.

In [3]:
BX_in_ORBIT= df.groupby("ORBIT_CNT")["BX_COUNTER"].max()
print(BX_in_ORBIT)

ORBIT_CNT
3869200167    3187
3869200168    3538
3869200169    2766
3869200170    3377
3869200171    3542
              ... 
3869200362    3425
3869200363    3460
3869200364    3352
3869200365    2787
3869200366    2522
Name: BX_COUNTER, Length: 200, dtype: int64


3\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition, and convert the new column to a Time Series.

*Hint:* introduce an offset to the absolute time such that the start of the data acquisition (i.e. the first entry) is zero.

In [4]:
# exercise 3 and 4
df['ABSOLUTE_TIME_NS'] = (df['ORBIT_CNT'] * df['BX_COUNTER'] * 25 + df['BX_COUNTER'] * 25 + df['TDC_MEAS'] * (25 / 30))
start_time = df['ABSOLUTE_TIME_NS'].min()
df['ABSOLUTE_TIME_NS'] -= start_time
df['ABSOLUTE_TIME'] = pd.to_datetime(df['ABSOLUTE_TIME_NS'], unit='ns')
print(df)

       HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS  \
0         1     0          123  3869200167        2374        26   
1         1     0          124  3869200167        2374        27   
2         1     0           63  3869200167        2553        28   
3         1     0           64  3869200167        2558        19   
4         1     0           64  3869200167        2760        25   
...     ...   ...          ...         ...         ...       ...   
19995     1     0           27  3869200366        2513        29   
19996     1     0           63  3869200366        2517         6   
19997     1     0           32  3869200366        2519         5   
19998     1     0           17  3869200366        2522        21   
19999     1     0           64  3869200366        2522         0   

       ABSOLUTE_TIME_NS                 ABSOLUTE_TIME  
0          2.296370e+14 1970-01-03 15:47:17.029970821  
1          2.296370e+14 1970-01-03 15:47:17.029970822  
2          2.46

4\. Find out the duration of the data taking in hours, minutes and seconds, by using the features of the Time Series. Perform this check reading the whole dataset.

5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

In [5]:
temp_channels = df.groupby('TDC_CHANNEL').size().reset_index(name='COUNT')
sorted_channels = temp_channels.sort_values(by='COUNT', ascending=False)
top_noisy_channels = sorted_channels.head(3)
print(top_noisy_channels)

     TDC_CHANNEL  COUNT
128          139   1693
63            64    996
62            63    969


6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit).

In [6]:
temp_orbits = df.groupby('ORBIT_CNT')['TDC_MEAS'].count().reset_index(name='COUNT2')
temp_orbits = temp_orbits[temp_orbits['COUNT2'] > 0]
non_empty_orbits = len(temp_orbits)
print("Number of non-empty orbits:", non_empty_orbits) 

Number of non-empty orbits: 200


7\. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

In [7]:
temp_orbits_139 = df[df['TDC_CHANNEL'] == 139]
unique_orbits_139 = temp_orbits_139['ORBIT_CNT'].nunique()
print("Number of unique orbits with at least one measurement from TDC_CHANNEL=139:", unique_orbits_139)

Number of unique orbits with at least one measurement from TDC_CHANNEL=139: 200


8\. Create two Series (one for each FPGA) that have the TDC channel as index, and the number of counts for the corresponding TDC channel as values.

In [8]:
fpga_0_data = df[df['FPGA'] == 0].groupby("TDC_CHANNEL").count()
fpga_1_data = df[df['FPGA'] == 1].groupby("TDC_CHANNEL").count()
fpga_0_data = fpga_0_data.drop(columns=["TDC_MEAS","BX_COUNTER","ORBIT_CNT","FPGA","ABSOLUTE_TIME","ABSOLUTE_TIME_NS"])
fpga_1_data = fpga_1_data.drop(columns=["TDC_MEAS","BX_COUNTER","ORBIT_CNT","FPGA","ABSOLUTE_TIME","ABSOLUTE_TIME_NS"])
fpga_0_data = fpga_0_data.rename(columns={'HEAD': 'count'})
fpga_1_data = fpga_1_data.rename(columns={'HEAD': 'count'})
print(fpga_0_data)
print(fpga_1_data)

             count
TDC_CHANNEL       
1               13
2               23
3               17
4               22
5               29
...            ...
121             98
122             94
123            280
124            258
139           1171

[118 rows x 1 columns]
             count
TDC_CHANNEL       
1              435
2              479
3              362
4              393
5              246
...            ...
125              7
126              9
127             21
128             25
139            522

[125 rows x 1 columns]


9\. **Optional:** Create two histograms (one for each FPGA) that show the number of counts for each TDC channel.